<a href="https://colab.research.google.com/github/Fardeen37/CodeAlpha_FAQs-Chatbot/blob/main/FAQs(Ai/ML)%20Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Installing required libraries
!pip install pandas sentence-transformers gradio

In [2]:
#Importing Libraries

import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
import gradio as gr
from google.colab import files

In [3]:
#Uploading FAQs Dataset in CSV Format

uploaded = files.upload()
faq_df = pd.read_csv(list(uploaded.keys())[0])
faq_df.head()

Saving faq_dataset.csv to faq_dataset.csv


,Question,Answer
0,What is Artificial Intelligence?,Artificial Intelligence is the simulation of h...
1,What is Machine Learning?,Machine Learning is a subset of AI that enable...
2,What is the difference between AI and ML?,AI is the broader concept of machines performi...
3,What are the main types of Machine Learning?,The three main types are supervised learning u...
4,What is supervised learning?,Supervised learning is a type of ML where mode...


In [4]:
#Loading Sentence Transformer Model
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
#Precomputing embeddings for all FAQ questions

faq_questions = faq_df['Question'].tolist()
faq_answers = faq_df['Answer'].tolist()
faq_embeddings = model.encode(faq_questions, convert_to_tensor=True)

In [6]:
#Chatbot logic (semantic search)

def chatbot_response(user_input):
    user_embedding = model.encode(user_input, convert_to_tensor=True)
    similarity = util.cos_sim(user_embedding, faq_embeddings)[0]
    idx = int(similarity.argmax())  # Best match index
    response = faq_answers[idx]
    return response

In [ ]:
#Building Chatbot UI with Gradio
with gr.Blocks(css=".gradio-container {background-color: #111; color: white; font-family: 'Arial';}") as demo:
    gr.Markdown("<h1 style='text-align:center; color:cyan;'>💬 AI-Powered FAQ Chatbot</h1>")

    chatbot = gr.Chatbot(height=400)
    msg = gr.Textbox(placeholder="Ask me anything...", label="Your Question")
    clear = gr.Button("Clear Chat")

    def respond(message, chat_history):
        response = chatbot_response(message)
        chat_history.append((message, response))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch()